# Sources de récupération de la donnée
## - APIs :
### Etendue géographique : https://apicarto.ign.fr/api/doc/
https://apicarto.ign.fr/api/doc/gpu#/Communes/get_gpu_municipality
### OSM data : 
https://www.youtube.com/watch?v=lxhIdsY1k38
https://wiki.openstreetmap.org/
https://overpass-turbo.eu/index.html
https://wiki.openstreetmap.org/wiki/FR:%C3%89l%C3%A9ments_cartographiques#Toponymie_(place)

## - Librairie OSPMnx :
https://pygis.io/docs/d_access_osm.html
https://osmnx.readthedocs.io/en/stable/user-reference.html#module-osmnx.features
https://github.com/gboeing/osmnx-examples/blob/main/notebooks/16-download-osm-geospatial-features.ipynb

<span style='color:red'> Plutôt utiliser OSMnx c'est plus facile ! </span>

# Fonction pour récupérer les données passage piétons OSM

In [2]:
#exemple d= 500 m
#point (X,Y) = (5.40280, 43.29729)

import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, box, polygon

def extract_from_bbox_osm(tags, S,N,E,W):

    """
    Extraire les données OSM à contenues dans un polygone.
    
    Cette fonction prend en entrée le polygone et le dictionnaire des tags d'OSM et retourne les données OSM à partir du polygone.
    Les tags OSM sont des paires clé-valeur décrivant les caractéristiques et les attributs des objets cartographiques (exmple: 'amenity' : 'restaurant')
    
    Paramètres :
    - tags (dict) : dictionnaire des tags utilisées pour rechercher des éléments d'OSM.
    - pol (shapely.geometry.Polygon) : polygone représentant les limites géographiques pour récupérer les entités dedans.
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame des données OSM identifiées à l'aide des tags et extraites à partir d'un polygone.
    """
    
    bbox=(S,N,E,W)
    
    data_bbox = ox.features_from_bbox(*bbox, tags = tags)
    
    return(data_bbox)

def get_bbox(S,N,E,W):

    """
    Créer le GeoDataFrame de la bbox. 
    
    Cette fonction prend en entrée les latitudes (N,S) et les longitude (E,W) de la bbox et retourne son le GeoDataFrame.
    
    Paramètres :
    - S (float) : Latitude min du centre de la zone tampon.
    - N (float) : Latitude max du centre de la zone tampon.
    - E (float) : Longitude min du centre de la zone tampon.
    - W (float) : Longitude max du centre de la zone tampon.
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame de la bbox.
    - (X,Y) (tuple) : Longitude et latitude du centre de la bbox.
    """
    bound=box(E,S,W,N) #création d'un polygone rectangle à partir des coordonnées des deux points de la bbox

    boundgdf = gpd.GeoDataFrame({'geometry': [bound]}, crs='EPSG:4326') #création du geodataframe du bbox

    #Coordonnées du centre de la bbox
    (X,Y)=((E+W)/2,(S+N)/2)
    
    return(boundgdf,(X,Y))
    
def extract_from_polygon_osm(tags, pol):

    """
    Extraire les données OSM à contenues dans un polygone.
    
    Cette fonction prend en entrée le polygone et le dictionnaire des tags d'OSM et retourne les données OSM à partir du polygone.
    Les tags OSM sont des paires clé-valeur décrivant les caractéristiques et les attributs des objets cartographiques (exmple: 'amenity' : 'restaurant')
    
    Paramètres :
    - tags (dict) : dictionnaire des tags utilisées pour rechercher des éléments d'OSM.
    - pol (shapely.geometry.Polygon) : polygone représentant les limites géographiques pour récupérer les entités dedans.
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame des données OSM identifiées à l'aide des tags et extraites à partir d'un polygone.
    """

    data_poly = ox.features_from_polygon(pol, tags = tags)

    return(data_poly)

def get_buffer(X, Y, d):

    """
    Créer une zone tampon à partir d'un point et d'une distance en mètres. 
    
    Cette fonction prend en entrée la latitude et la longitude du centre de la zone tampon ainsi que
    la distance par rapport au centre et retourne la zone tampon sous format "GeoDataFrame"
    
    Paramètres :
    - X (float) : Latitude du centre de la zone tampon.
    - Y (float) : Longitude du centre de la zone tampon.
    - d (float) : Distance par rapport au centre de la zone tampon.
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame de la zone tampon.
    """

    X = float(X)
    Y = float(Y)
    d = float(d)
    
    point = Point(X,Y)
    
    gpdpoint = gpd.GeoDataFrame({'geometry': [point]}, crs="EPSG:4326").to_crs(epsg = 2154) #pour prendre en concidération la distance en mètres avant la création de la zone tampon
        
    buffered = gpdpoint.buffer(d, resolution=20).to_crs(epsg = 4326)
        
    gdf_buffered = gpd.GeoDataFrame(geometry = buffered)
    
    return(gdf_buffered)

def extract_from_point_osm(tags, X, Y, d):

    """
    Extraire les données OSM contenues dans une zone tampon.
    
    Cette fonction prend en entrée les coordonnées, la distance de la zone tampon ainsi que le dictionnaire des tags d'OSM et retourne
    les données OSM incluses dans cette zone.
    
    Paramètres :
    - X (float) : Latitude du centre de la zone tampon.
    - Y (float) : Longitude du centre de la zone tampon.
    - d (float) : Distance par rapport au centre de la zone tampon.
    - tags (dict) : dictionnaire des tags utilisées pour rechercher des éléments d'OSM.
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame des données OSM identifiées à l'aide des tags et extraites à partir d'un polygone.
    """

    buffer = get_buffer(X, Y, d)
    pol = buffer['geometry'].iloc[0] #avoir le polygone de la zone tampon
    data_buffer = extract_from_polygon_osm(tags, pol)

    return(data_buffer)

def export_osm(data, output):

    """
    Exporter les données OSM extraites vers un GeoPackage.
    
    Cette procédure prend en entrée les données OSM extraites et le schéma du fichier de sortie.
    
    Paramètres :
    - data (geopandas.GeoDataFrame) : Le GeoDataFrame des données OSM extraites.
    - output (string) : Schéma du fichier de sortie en GeoPackage 
    """    
    
    # Conversion en String pour que l'export fonctionne
    data = data.map(lambda x: x if not isinstance(x, list) else str(x))

    # Export en GeoPackage
    data.to_file(output, driver = "GPKG")
    
def get_passage_pieton_buffer_OSM(X, Y, d, output):

    """
    Récupération des données OSM catégorie "passage piétons" à partir d'une zone tompon.
    
    Cette fonction prend en entrée les coordonnées, la distance de la zone tampon ainsi que le schéma du fichier de sortie en GeoPackage et retourne
    les données OSM catégorie "passage piétons" incluses dans cette zone.
    
    Paramètres :
    - X (float) : Latitude du centre de la zone tampon.
    - Y (float) : Longitude du centre de la zone tampon.
    - d (float) : Distance par rapport au centre de la zone tampon.
    - output (string) : Schéma du fichier de sortie en GeoPackage
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame des données OSM "passage piétons" extraites à partir d'une zone tampon.
    """

    # Voici les tags OSM associés aux passages piétons
    tags = {'highway': 'crossing'}

    # Extraction des données OSM relatives aux passages piétons
    passage_pieton = extract_from_point_osm(tags, X, Y, d)

    # Export des données en Geopackage
    export_osm(passage_pieton, output)
    
    return(passage_pieton)

def get_passage_pieton_bbox_OSM(S,N,E,W, output):

    """
    Récupération des données OSM catégorie "passage piétons" à partir d'une bbox.
    
    Cette fonction prend en entrée la bbox ainsi que le schéma du fichier de sortie en GeoPackage et retourne
    les données OSM catégorie "passage piétons" incluses dans cette bbox.
    
    Paramètres :
    - S (float) : Latitude min du centre de la zone tampon.
    - N (float) : Latitude max du centre de la zone tampon.
    - E (float) : Longitude min du centre de la zone tampon.
    - W (float) : Longitude max du centre de la zone tampon. 
    - output (string) : Schéma du fichier de sortie en GeoPackage
    
    Retour :
    - geopandas.GeoDataFrame : Le GeoDataFrame des données OSM "passage piétons" extraites à partir d'une bbox.
    """
    
    # Voici les tags OSM associés aux passages piétons
    tags = {'highway': 'crossing'}

    # Extraction des données OSM relatives aux passages piétons
    passage_pieton = extract_from_bbox_osm(tags, S,N,E,W)
    
    # Export des données en Geopackage
    export_osm(passage_pieton, output)
    
    return(passage_pieton)


# Application buffer:
#(X,Y,d) = (5.40280, 43.29729, 500)
#output = "../open_data/processed/passage_pieton_buffer_OSM.gpkg"
#passage_pieton = get_passage_pieton_buffer_OSM(X, Y, d, output)

#Application bbox:
(S,N,E,W) = (43.295426, 43.317027, 5.389065, 5.411281)
output = "../open_data/processed/passage_pieton_bbox_OSM.gpkg"
passage_pieton = get_passage_pieton_bbox_OSM(S,N,E,W, output)

C:\Users\alaeddine.jerad\AppData\Local\Temp\ipykernel_5048\985596567.py:26: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  data_bbox = ox.features_from_bbox(*bbox, tags = tags)


In [3]:
# Visualisation :
from ipyleaflet import Map, GeoJSON, GeoData, ZoomControl

import geopandas, pandas as pd, numpy as np

def render_map(data, pol):

    """
    Visualsation des données OSM extraites à partir d'un polygone quelconque (bbox ou buffer ou contour de commune/quartier...).
    
    Cette procedure prend en entrée les données OSM ainsi que le polygone à partir duquel ces données sont extraites et retourne
    les données OSM catégorie "transport" incluses dans cette zone.
    
    Paramètres :
    - data (geopandas.GeoDataFrame) : Le GeoDataFrame des données OSM extraites.
    - pol (geopandas.GeoDataFrame) : Le GeoDataFrame du polygone d'extraction. 
    """    
    
    m = Map(center=(Y,X), zoom=14)

    # Visualisation des données extraites
    geo_data1 = GeoData(geo_dataframe = data,
        style={'color': 'black', 'radius':3, 'fillColor': '#cc3333', 'opacity':1, 'weight':1, 'dashArray':'2', 'fillOpacity':1},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Release')

    # Visualisation du polygone d'extraction
    geo_data2 = GeoData(geo_dataframe = pol,
        style={'color': 'blue', 'radius':2, 'fillColor': '#004dff', 'opacity':1, 'weight':2, 'dashArray':'1', 'fillOpacity':0})
    
    m.add(geo_data2)
    m.add(geo_data1)

    m.fit_bounds([[pol.bounds['miny'].values[0], pol.bounds['minx'].values[0]], # Ajuster le zoom et centrer la carte pour afficher les limites du polygone
              [pol.bounds['maxy'].values[0], pol.bounds['maxx'].values[0]]])
    
    display(m)

#Application buffer:
#(X,Y,d) = (5.40280, 43.29729, 500)
#pol = get_buffer(X, Y, d)
#render_map(passage_pieton, pol)

#Application bbox:
(S,N,E,W) = (43.295426, 43.317027, 5.389065, 5.411281)
pol2,(X,Y) = get_bbox(S,N,E,W)
render_map(passage_pieton, pol2)

Map(center=[43.3062265, 5.400173000000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

In [4]:
passage_pieton

button_operated         crossing crossing_ref  \
element_type osmid                                                       
node         12165069                yes  traffic_signals        zebra   
             33781291                yes  traffic_signals        zebra   
             267217883               NaN     uncontrolled        zebra   
             269367670               NaN     uncontrolled        zebra   
             304963513               NaN     uncontrolled        zebra   
...                                  ...              ...          ...   
             12076997060             NaN     uncontrolled          NaN   
             12076997061             NaN     uncontrolled          NaN   
             12076997062             NaN     uncontrolled          NaN   
             12076997064             NaN     uncontrolled          NaN   
             12083606744             NaN     uncontrolled          NaN   

                           highway tactile_paving traffic_signals:sound  \
element_type osmid                                                        
node         12165069     crossing            yes                    no   
             33781291     crossing            yes                   NaN   
             267217883    crossing            yes                   NaN   
             269367670    crossing            yes                   NaN   
             304963513    crossing            NaN                   NaN   
...                            ...            ...                   ...   
             12076997060  crossing             no                   NaN   
             12076997061  crossing             no                   NaN   
             12076997062  crossing            yes                   NaN   
             12076997064  crossing            yes                   NaN   
             12083606744  crossing            yes                   NaN   

                         traffic_signals:vibration                  geometry  \
element_type osmid                                                             
node         12165069                          yes  POINT (5.40089 43.30670)   
             33781291                          NaN  POINT (5.40083 43.29762)   
             267217883                         NaN  POINT (5.40553 43.31258)   
             269367670                         NaN  POINT (5.40203 43.29673)   
             304963513                         NaN  POINT (5.40577 43.29705)   
...                                            ...                       ...   
             12076997060                       NaN  POINT (5.39635 43.30087)   
             12076997061                       NaN  POINT (5.39624 43.30090)   
             12076997062                       NaN  POINT (5.39605 43.30147)   
             12076997064                       NaN  POINT (5.39598 43.30148)   
             12083606744                       NaN  POINT (5.39601 43.30251)   

                         crossing:island     kerb source bicycle  \
element_type osmid                                                 
node         12165069                NaN      NaN    NaN     NaN   
             33781291                NaN      NaN    NaN     NaN   
             267217883               NaN      NaN    NaN     NaN   
             269367670                no  lowered    NaN     NaN   
             304963513               NaN      NaN    NaN     NaN   
...                                  ...      ...    ...     ...   
             12076997060             NaN      NaN    NaN     NaN   
             12076997061             NaN      NaN    NaN     NaN   
             12076997062             NaN      NaN    NaN     NaN   
             12076997064             NaN      NaN    NaN     NaN   
             12083606744             NaN      NaN    NaN     NaN   

                         crossing:markings check_date:crossing wheelchair  \
element_type osmid                                                          
node  